In [33]:
print("Hello, World!")

Hello, World!


In [34]:
from langchain_groq import ChatGroq
import os

In [35]:
from dotenv import load_dotenv
load_dotenv()

True

In [36]:
 GROQ_API_KEY=os.environ.get("GROQ_API_KEY")
model= ChatGroq(model="qwen/qwen3-32b", api_key=GROQ_API_KEY)

In [37]:
model=ChatGroq(model="qwen/qwen3-32b")

In [38]:
model.invoke("What is the capital of France?")

AIMessage(content='<think>\nOkay, so the user is asking, "What is the capital of France?" Hmm, let me think. I remember from school that France is a country in Europe, and its capital is a major city. Let me recall... I think it\'s Paris. Wait, isn\'t Paris known for the Eiffel Tower and the Louvre? Yeah, that sounds right. Let me double-check to make sure I\'m not mixing it up with another country. For example, Spain\'s capital is Madrid, Germany\'s is Berlin, Italy\'s is Rome. So France\'s should be Paris. I don\'t think there\'s any confusion there. Maybe they just want a straightforward answer. I should confirm if there are any other possible answers, but I\'m pretty confident it\'s Paris. Yeah, I\'ll go with Paris as the capital of France.\n</think>\n\nThe capital of France is **Paris**. Known for iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral, Paris is one of the most famous cities in the world and a major cultural, historical, and politica

In [39]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [40]:
embedding_model=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [92]:
dummy=embedding_model.embed_query("What is the capital of France?")

## 1. Data INgestion

In [42]:
from langchain.document_loaders import PyPDFLoader

In [43]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [44]:
os.path.join(os.getcwd(), "data", "sample.pdf")

'c:\\Users\\sharma.rakesh\\Desktop\\Projects\\LLM_ops\\document_portal\\notebook\\data\\sample.pdf'

In [45]:
os.getcwd()

'c:\\Users\\sharma.rakesh\\Desktop\\Projects\\LLM_ops\\document_portal\\notebook'

In [47]:
loader=PyPDFLoader(os.path.join(os.getcwd(), "data", "sample.pdf"))

In [49]:
documents=loader.load()

In [50]:
len(documents)

77

In [ ]:
## there is no determnitic way to split the text,we can try different chunk sizes and overlap
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, 
                                             chunk_overlap=200,length_function=len)

In [53]:
docs=text_splitter.split_documents(documents)

In [54]:
len(docs)

488

In [56]:
docs[0].metadata

{'producer': 'pdfTeX-1.40.25',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2023-07-20T00:30:36+00:00',
 'author': '',
 'keywords': '',
 'moddate': '2023-07-20T00:30:36+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
 'subject': '',
 'title': '',
 'trapped': '/False',
 'source': 'c:\\Users\\sharma.rakesh\\Desktop\\Projects\\LLM_ops\\document_portal\\notebook\\data\\sample.pdf',
 'total_pages': 77,
 'page': 0,
 'page_label': '1'}

In [93]:
docs[0].page_content

'Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗ Louis Martin† Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev\nPunit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich\nYinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra\nIgor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi\nAlan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang\nRoss Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang\nAngela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic'

In [94]:
check=embedding_model.embed_documents([docs[0].page_content])

In [63]:
len(embedding_model.embed_documents([docs[0].page_content][0]))

986

In [82]:
from langchain.vectorstores import FAISS

In [85]:
## in memory vector store
vectorstore=FAISS.from_documents(docs, embedding_model)

## 2. Retrrieval processes 

In [95]:
relevant_doc=vectorstore.similarity_search("What is a llama2 llm model")

In [96]:
relevant_doc[0]

Document(id='7738d961-0da7-40ab-97f2-29f8676a30b5', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'c:\\Users\\sharma.rakesh\\Desktop\\Projects\\LLM_ops\\document_portal\\notebook\\data\\sample.pdf', 'total_pages': 77, 'page': 73, 'page_label': '74'}, page_content='Llama 1\n7B 0.27 0.26 0.34 0.54 0.36 0.39 0.26 0.28 0.33 0.45 0.33 0.17 0.24 0.31 0.44 0.57 0.39 0.3513B 0.24 0.24 0.31 0.52 0.37 0.37 0.23 0.28 0.31 0.50 0.27 0.10 0.24 0.27 0.41 0.55 0.34 0.2533B 0.23 0.26 0.34 0.50 0.36 0.35 0.24 0.33 0.34 0.49 0.31 0.12 0.23 0.30 0.41 0.60 0.28 0.2765B 0.25 0.26 0.34 0.46 0.36 0.40 0.25 0.32 0.32 0.48 0.31 0.11 0.25 0.30 0.43 0.60 0.39 0.34\nLlama 2\n7B 0.28 0.25 0.29 0.50 0.36 0.37 

In [97]:
print(relevant_doc[0].page_content)

Llama 1
7B 0.27 0.26 0.34 0.54 0.36 0.39 0.26 0.28 0.33 0.45 0.33 0.17 0.24 0.31 0.44 0.57 0.39 0.3513B 0.24 0.24 0.31 0.52 0.37 0.37 0.23 0.28 0.31 0.50 0.27 0.10 0.24 0.27 0.41 0.55 0.34 0.2533B 0.23 0.26 0.34 0.50 0.36 0.35 0.24 0.33 0.34 0.49 0.31 0.12 0.23 0.30 0.41 0.60 0.28 0.2765B 0.25 0.26 0.34 0.46 0.36 0.40 0.25 0.32 0.32 0.48 0.31 0.11 0.25 0.30 0.43 0.60 0.39 0.34
Llama 2
7B 0.28 0.25 0.29 0.50 0.36 0.37 0.21 0.34 0.32 0.50 0.28 0.19 0.26 0.32 0.44 0.51 0.30 0.2513B 0.24 0.25 0.35 0.50 0.41 0.36 0.24 0.39 0.35 0.48 0.31 0.18 0.27 0.34 0.46 0.66 0.35 0.2834B 0.27 0.24 0.33 0.56 0.41 0.36 0.26 0.32 0.36 0.53 0.33 0.07 0.26 0.30 0.45 0.56 0.26 0.3570B 0.31 0.29 0.35 0.51 0.41 0.45 0.27 0.34 0.40 0.52 0.36 0.12 0.28 0.31 0.45 0.65 0.33 0.20
Fine-tuned
